# TRAIN Gray

In [11]:
import cv2
import numpy as np
import os
import csv
import datetime

def load_data(dataset_path='Datasets/numbers/mnist_png/Hnd'):
    """
    Fungsi untuk memuat data dari folder dataset.
    Setiap folder (Sample0, Sample1, …, Sample9) mewakili kelas angka.
    Citra dibaca dalam mode grayscale, diflatten dan dinormalisasi.
    """
    X = []
    y = []
    # Lakukan iterasi untuk setiap kelas digit 0-9
    for digit in range(10):
        folder = os.path.join(dataset_path, f"Sample{digit}")
        # Pastikan folder ada
        if not os.path.isdir(folder):
            print(f"Folder {folder} tidak ditemukan.")
            continue
        for filename in os.listdir(folder):
            if filename.endswith('.png'):
                filepath = os.path.join(folder, filename)
                # Baca gambar dalam grayscale
                img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    continue
                # Flatten citra menjadi vektor 1D dan normalisasi nilai piksel ke [0,1]
                img_flat = img.flatten().astype(np.float32) / 255.0
                X.append(img_flat)
                y.append(digit)
    X = np.array(X)
    y = np.array(y)
    print(f"img shape {img.shape}")
    print(f"img flat : {X[0].shape}")
    return X, y

def one_hot_encode(y, num_classes=10):
    """
    Mengubah label y menjadi format one-hot.
    """
    one_hot = np.zeros((y.shape[0], num_classes))
    # print("One hot before: ", one_hot)
    one_hot[np.arange(y.shape[0]), y] = 1
    # print("One hot after: ", one_hot)
    return one_hot



In [12]:
def softmax(z):
    """
    Fungsi softmax untuk mengubah output linear menjadi probabilitas.
    Dikurangi nilai maksimum tiap baris untuk kestabilan numerik.
    """
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    # print(f"exp_z: {exp_z} np.max: {np.max(z, axis=1, keepdims=True)} np.sum: {np.sum(exp_z, axis=1, keepdims=True)}") 
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def save_model(model, filename="model_gray.npz"):
    """
    Menyimpan model ke file NPZ.
    Parameter yang disimpan adalah weight (W) dan bias (b).
    """
    np.savez(filename, W=model['W'], b=model['b'])
    print(f"Model disimpan ke file {filename}")
    
def load_model(filename="model.npz"):
    """
    Memuat model dari file NPZ.
    Mengembalikan dictionary dengan weight (W), bias (b),
    dan fungsi net serta netfunction untuk prediksi.
    """
    data = np.load(filename)
    W = data['W']
    b = data['b']
    
    def net(x):
        return np.dot(x, W) + b

    def netfunction(x):
        # Pastikan fungsi softmax sudah didefinisikan
        return softmax(net(x))
    
    model = {
        'W': W,
        'b': b,
        'net': net,
        'netfunction': netfunction
    }
    print(f"Model berhasil dimuat dari file {filename}")
    return model

def train_nn(X, y, learning_rate=0.01, epochs=100):
    """
    Fungsi training untuk model klasifikasi:
      - Inisialisasi bobot dan bias
      - Lakukan forward pass (net = X.W + b) dan hitung softmax (fnet)
      - Hitung loss cross-entropy
      - Hitung gradien (dz, dw, db) dengan backpropagation
      - Update parameter menggunakan gradient descent
      
    Log training per epoch disimpan ke file CSV dengan kolom:
      Epoch, X, Y_Label, W, b, net, fnet, dw, dz, db
      
    Note: Untuk menghindari log data yang terlalu besar, hanya nilai sample pertama (index 0)
    yang dijadikan representasi.
    """
    n_samples, n_features = X.shape
    print(f"n_samples: {n_samples}, n_features: {n_features}")
    n_classes = 10

    # Inisialisasi bobot dan bias
    W = np.random.randn(n_features, n_classes) * 0.01
    b = np.zeros((1, n_classes))

    # Ubah label ke format one-hot
    y_encoded = one_hot_encode(y, n_classes)
    # print(f"y_encoded: {y_encoded}")
    # print(f"y_encoded shape: {y_encoded.shape}")

    # List untuk menyimpan log per epoch
    log_data = []

    for epoch in range(epochs):
        # Forward pass
        Z = np.dot(X, W) + b       # net input
        A = softmax(Z)             # fnet, output probabilitas

        # Hitung loss cross-entropy
        loss = -np.sum(y_encoded * np.log(A + 1e-8)) / n_samples

        # Backpropagation
        dZ = A - y_encoded         # error (dz)
        dW = np.dot(X.T, dZ) / n_samples   # gradien bobot
        db = np.sum(dZ, axis=0, keepdims=True) / n_samples  # gradien bias

        # Update parameter
        W -= learning_rate * dW
        b -= learning_rate * db

        # Simpan log untuk epoch ini, ambil nilai sample pertama sebagai contoh
        log_data.append({
            "Epoch": epoch,
            "X": np.array2string(X[0], precision=2, separator=','),
            "Y_Label": y[0],
            "W": np.array2string(W, precision=4, separator=','),
            "b": np.array2string(b, precision=4, separator=','),
            "net": np.array2string(Z[0], precision=4, separator=','),
            "fnet": np.array2string(A[0], precision=4, separator=','),
            "dw": np.array2string(dW, precision=4, separator=','),
            "dz": np.array2string(dZ[0], precision=4, separator=','),
            "db": np.array2string(db, precision=4, separator=',')
        })

        if epoch % 10 == 0:
            print(f"Epoch {epoch}/{epochs}, Loss: {loss:.4f}")

    # Simpan log training ke file CSV
    # Tambahkan timestamp ke nama file log
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    csv_filename = f"training_log_detailed_{timestamp}.csv"

    with open(csv_filename, 'w', newline='') as csvfile:
        fieldnames = ["Epoch", "X", "Y_Label", "W", "b", "net", "fnet", "dw", "dz", "db"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in log_data:
            writer.writerow(row)
    print(f"Detailed log training disimpan ke file {csv_filename}")

    # Definisikan fungsi net dan netfunction untuk keperluan prediksi
    def net(x):
        return np.dot(x, W) + b

    def netfunction(x):
        z = net(x)
        return softmax(z)

    # Kembalikan model
    model = {
        'W': W,
        'b': b,
        'net': net,
        'netfunction': netfunction
    }
    return model



In [ ]:
def main():
    # Memuat data dari folder dataset
    X, y = load_data()
    print("Data berhasil dimuat. Shape X:", X.shape, ", Shape y:", y.shape)

    # Training model dengan parameter tertentu (misal: learning_rate=0.1, epochs=100)
    model = train_nn(X, y, learning_rate=0.1, epochs=100)

    # Contoh prediksi untuk data pertama
    sample = X[0].reshape(1, -1)
    net_value = model['net'](sample)           # Nilai linear (net)
    fnet_value = model['netfunction'](sample)    # Output softmax (fnet)
    predicted_class = np.argmax(fnet_value)
    save_model(model, "model_gray.npz")
    print("Prediksi kelas untuk sample pertama:", predicted_class)
    print("Bias shape: ", model['b'].shape)
    print("Weight shape: ", model['W'].shape)

    # Simpan nilai prediksi (net dan fnet) ke file teks
    with open("prediction_details.txt", "w") as f:
        f.write("Prediksi untuk sample pertama:\n")
        f.write(f"Predicted Class: {predicted_class}\n\n")
        f.write("Net (nilai linear):\n")
        f.write(np.array2string(net_value, precision=4, separator=', ') + "\n\n")
        f.write("fnet (hasil softmax):\n")
        f.write(np.array2string(fnet_value, precision=4, separator=', ') + "\n")
        # Write nilai weight dan bias
        f.write("\nWeight:\n")
        f.write(np.array2string(model['W'], precision=4, separator=', ') + "\n\n")
        f.write("Bias:\n")
        f.write(np.array2string(model['b'], precision=4, separator=', ') + "\n")
    print("Detail prediksi disimpan ke file prediction_details.txt")

if __name__ == "__main__":
    main()


img shape (28, 28)
img flat : (784,)
Data berhasil dimuat. Shape X: (60000, 784) , Shape y: (60000,)
n_samples: 60000, n_features: 784
Epoch 0/100, Loss: 2.3114
Epoch 10/100, Loss: 1.5491
Epoch 20/100, Loss: 1.1872
Epoch 30/100, Loss: 0.9937
Epoch 40/100, Loss: 0.8755
Epoch 50/100, Loss: 0.7959
Epoch 60/100, Loss: 0.7383
Epoch 70/100, Loss: 0.6945
Epoch 80/100, Loss: 0.6600
Epoch 90/100, Loss: 0.6319
Detailed log training disimpan ke file training_log_detailed_20250225_184816.csv
Model disimpan ke file model.npz
Prediksi kelas untuk sample pertama: 0
Bias shape:  (1, 10)
Weight shape:  (784, 10)
Detail prediksi disimpan ke file prediction_details.txt


# TRAIN COLOR

In [7]:
import cv2
import numpy as np
import os
import csv
import datetime

def load_data(dataset_path='Datasets/numbers/mnist_png/Hnd'):
    """
    Fungsi untuk memuat data dari folder dataset.
    Setiap folder (Sample0, Sample1, …, Sample9) mewakili kelas angka.
    Citra dibaca dalam mode grayscale, diflatten dan dinormalisasi.
    """
    X = []
    y = []
    # Lakukan iterasi untuk setiap kelas digit 0-9
    for digit in range(10):
        folder = os.path.join(dataset_path, f"Sample{digit}")
        # Pastikan folder ada
        if not os.path.isdir(folder):
            print(f"Folder {folder} tidak ditemukan.")
            continue
        for filename in os.listdir(folder):
            if filename.endswith('.png'):
                filepath = os.path.join(folder, filename)
                # Baca gambar dalam grayscale
                img = cv2.imread(filepath)
                if img is None:
                    continue
                # Flatten citra menjadi vektor 1D dan normalisasi nilai piksel ke [0,1]
                img_flat = img.flatten().astype(np.float32) / 255.0
                X.append(img_flat)
                y.append(digit)
    X = np.array(X)
    y = np.array(y)
    print(f"img shape {img.shape}")
    print(f"img flat : {X[0].shape}")
    return X, y

def one_hot_encode(y, num_classes=10):
    """
    Mengubah label y menjadi format one-hot.
    """
    one_hot = np.zeros((y.shape[0], num_classes))
    # print("One hot before: ", one_hot)
    one_hot[np.arange(y.shape[0]), y] = 1
    # print("One hot after: ", one_hot)
    return one_hot



In [8]:
def softmax(z):
    """
    Fungsi softmax untuk mengubah output linear menjadi probabilitas.
    Dikurangi nilai maksimum tiap baris untuk kestabilan numerik.
    """
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    # print(f"exp_z: {exp_z} np.max: {np.max(z, axis=1, keepdims=True)} np.sum: {np.sum(exp_z, axis=1, keepdims=True)}") 
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def save_model(model, filename="model_color.npz"):
    """
    Menyimpan model ke file NPZ.
    Parameter yang disimpan adalah weight (W) dan bias (b).
    """
    np.savez(filename, W=model['W'], b=model['b'])
    print(f"Model disimpan ke file {filename}")
    
def load_model(filename="model.npz"):
    """
    Memuat model dari file NPZ.
    Mengembalikan dictionary dengan weight (W), bias (b),
    dan fungsi net serta netfunction untuk prediksi.
    """
    data = np.load(filename)
    W = data['W']
    b = data['b']
    
    def net(x):
        return np.dot(x, W) + b

    def netfunction(x):
        # Pastikan fungsi softmax sudah didefinisikan
        return softmax(net(x))
    
    model = {
        'W': W,
        'b': b,
        'net': net,
        'netfunction': netfunction
    }
    print(f"Model berhasil dimuat dari file {filename}")
    return model

def train_nn(X, y, learning_rate=0.01, epochs=100):
    """
    Fungsi training untuk model klasifikasi:
      - Inisialisasi bobot dan bias
      - Lakukan forward pass (net = X.W + b) dan hitung softmax (fnet)
      - Hitung loss cross-entropy
      - Hitung gradien (dz, dw, db) dengan backpropagation
      - Update parameter menggunakan gradient descent
      
    Log training per epoch disimpan ke file CSV dengan kolom:
      Epoch, X, Y_Label, W, b, net, fnet, dw, dz, db
      
    Note: Untuk menghindari log data yang terlalu besar, hanya nilai sample pertama (index 0)
    yang dijadikan representasi.
    """
    n_samples, n_features = X.shape
    print(f"n_samples: {n_samples}, n_features: {n_features}")
    n_classes = 10

    # Inisialisasi bobot dan bias
    W = np.random.randn(n_features, n_classes) * 0.01
    b = np.zeros((1, n_classes))

    # Ubah label ke format one-hot
    y_encoded = one_hot_encode(y, n_classes)
    # print(f"y_encoded: {y_encoded}")
    # print(f"y_encoded shape: {y_encoded.shape}")

    # List untuk menyimpan log per epoch
    log_data = []

    for epoch in range(epochs):
        # Forward pass
        Z = np.dot(X, W) + b       # net input
        A = softmax(Z)             # fnet, output probabilitas

        # Hitung loss cross-entropy
        loss = -np.sum(y_encoded * np.log(A + 1e-8)) / n_samples

        # Backpropagation
        dZ = A - y_encoded         # error (dz)
        dW = np.dot(X.T, dZ) / n_samples   # gradien bobot
        db = np.sum(dZ, axis=0, keepdims=True) / n_samples  # gradien bias

        # Update parameter
        W -= learning_rate * dW
        b -= learning_rate * db

        # Simpan log untuk epoch ini, ambil nilai sample pertama sebagai contoh
        log_data.append({
            "Epoch": epoch,
            "X": np.array2string(X[0], precision=2, separator=','),
            "Y_Label": y[0],
            "W": np.array2string(W, precision=4, separator=','),
            "b": np.array2string(b, precision=4, separator=','),
            "net": np.array2string(Z[0], precision=4, separator=','),
            "fnet": np.array2string(A[0], precision=4, separator=','),
            "dw": np.array2string(dW, precision=4, separator=','),
            "dz": np.array2string(dZ[0], precision=4, separator=','),
            "db": np.array2string(db, precision=4, separator=',')
        })

        if epoch % 10 == 0:
            print(f"Epoch {epoch}/{epochs}, Loss: {loss:.4f}")

    # Simpan log training ke file CSV
    # Tambahkan timestamp ke nama file log
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    csv_filename = f"training_log_detailed_{timestamp}.csv"

    with open(csv_filename, 'w', newline='') as csvfile:
        fieldnames = ["Epoch", "X", "Y_Label", "W", "b", "net", "fnet", "dw", "dz", "db"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in log_data:
            writer.writerow(row)
    print(f"Detailed log training disimpan ke file {csv_filename}")

    # Definisikan fungsi net dan netfunction untuk keperluan prediksi
    def net(x):
        return np.dot(x, W) + b

    def netfunction(x):
        z = net(x)
        return softmax(z)

    # Kembalikan model
    model = {
        'W': W,
        'b': b,
        'net': net,
        'netfunction': netfunction
    }
    return model



In [9]:
def main():
    # Memuat data dari folder dataset
    X, y = load_data()
    print("Data berhasil dimuat. Shape X:", X.shape, ", Shape y:", y.shape)

    # Training model dengan parameter tertentu (misal: learning_rate=0.1, epochs=100)
    model = train_nn(X, y, learning_rate=0.1, epochs=100)

    # Contoh prediksi untuk data pertama
    sample = X[0].reshape(1, -1)
    net_value = model['net'](sample)           # Nilai linear (net)
    fnet_value = model['netfunction'](sample)    # Output softmax (fnet)
    predicted_class = np.argmax(fnet_value)
    save_model(model, "model.npz")
    print("Prediksi kelas untuk sample pertama:", predicted_class)
    print("Bias shape: ", model['b'].shape)
    print("Weight shape: ", model['W'].shape)

    # Simpan nilai prediksi (net dan fnet) ke file teks
    with open("prediction_details.txt", "w") as f:
        f.write("Prediksi untuk sample pertama:\n")
        f.write(f"Predicted Class: {predicted_class}\n\n")
        f.write("Net (nilai linear):\n")
        f.write(np.array2string(net_value, precision=4, separator=', ') + "\n\n")
        f.write("fnet (hasil softmax):\n")
        f.write(np.array2string(fnet_value, precision=4, separator=', ') + "\n")
        # Write nilai weight dan bias
        f.write("\nWeight:\n")
        f.write(np.array2string(model['W'], precision=4, separator=', ') + "\n\n")
        f.write("Bias:\n")
        f.write(np.array2string(model['b'], precision=4, separator=', ') + "\n")
    print("Detail prediksi disimpan ke file prediction_details.txt")

if __name__ == "__main__":
    main()


img shape (28, 28, 3)
img flat : (2352,)
Data berhasil dimuat. Shape X: (60000, 2352) , Shape y: (60000,)
n_samples: 60000, n_features: 2352
Epoch 0/100, Loss: 2.3114
Epoch 10/100, Loss: 0.9870
Epoch 20/100, Loss: 0.7357
Epoch 30/100, Loss: 0.6310


KeyboardInterrupt: 

# Train 100 x 100 Color

In [2]:
import cv2
import numpy as np
import os
import csv
import datetime
import pandas as pd

def load_data(dataset_path='Datasets/SizeSeratus/numbers/mnist_png'):
    X = []
    y = []
    # Lakukan iterasi untuk setiap kelas digit 0-9
    for digit in range(10):
        folder = os.path.join(dataset_path, f"Sample{digit}")
        # Pastikan folder ada
        if not os.path.isdir(folder):
            print(f"Folder {folder} tidak ditemukan.")
            continue
        for filename in os.listdir(folder):
            if filename.endswith('.png'):
                filepath = os.path.join(folder, filename)
                # Baca gambar dalam grayscale
                img = cv2.imread(filepath)
                if img is None:
                    continue
                # Flatten citra menjadi vektor 1D dan normalisasi nilai piksel ke [0,1]
                img_flat = img.flatten().astype(np.float32) / 255.0
                X.append(img_flat)
                y.append(digit)
    X = np.array(X)
    y = np.array(y)
    print(f"img shape {img.shape}")
    print(f"img flat : {X[0].shape}")
    return X, y

def one_hot_encode(y, num_classes=10):
    """
    Mengubah label y menjadi format one-hot.
    """
    one_hot = np.zeros((y.shape[0], num_classes))
    # print("One hot before: ", one_hot)
    one_hot[np.arange(y.shape[0]), y] = 1
    # print("One hot after: ", one_hot)
    y = np.array(y)
    # Dataframe y dan y encoded comparison
    df = pd.DataFrame(data=y, columns=['y'])
    df2 = pd.DataFrame(data=one_hot, columns=['y0', 'y1', 'y2', 'y3', 'y4', 'y5', 'y6', 'y7', 'y8', 'y9'])
    df = pd.concat([df, df2], axis=1)
    print(df)
    return one_hot




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\agusf\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\agusf\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\agusf\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_lo

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\agusf\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\agusf\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\agusf\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_lo

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\agusf\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\agusf\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\agusf\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_lo

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\agusf\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\agusf\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\agusf\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_lo

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



In [3]:
def softmax(z):
    """
    Fungsi softmax untuk mengubah output linear menjadi probabilitas.
    Dikurangi nilai maksimum tiap baris untuk kestabilan numerik.
    """
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    # print(f"exp_z: {exp_z} np.max: {np.max(z, axis=1, keepdims=True)} np.sum: {np.sum(exp_z, axis=1, keepdims=True)}") 
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def save_model(model, filename="model_color.npz"):
    """
    Menyimpan model ke file NPZ.
    Parameter yang disimpan adalah weight (W) dan bias (b).
    """
    np.savez(filename, W=model['W'], b=model['b'])
    print(f"Model disimpan ke file {filename}")
    
def load_model(filename="model.npz"):
    """
    Memuat model dari file NPZ.
    Mengembalikan dictionary dengan weight (W), bias (b),
    dan fungsi net serta netfunction untuk prediksi.
    """
    data = np.load(filename)
    W = data['W']
    b = data['b']
    
    def net(x):
        return np.dot(x, W) + b

    def netfunction(x):
        # Pastikan fungsi softmax sudah didefinisikan
        return softmax(net(x))
    
    model = {
        'W': W,
        'b': b,
        'net': net,
        'netfunction': netfunction
    }
    print(f"Model berhasil dimuat dari file {filename}")
    return model

def train_nn(X, y, learning_rate=0.01, epochs=100):
    """
    Fungsi training untuk model klasifikasi:
      - Inisialisasi bobot dan bias
      - Lakukan forward pass (net = X.W + b) dan hitung softmax (fnet)
      - Hitung loss cross-entropy
      - Hitung gradien (dz, dw, db) dengan backpropagation
      - Update parameter menggunakan gradient descent
      
    Log training per epoch disimpan ke file CSV dengan kolom:
      Epoch, X, Y_Label, W, b, net, fnet, dw, dz, db
      
    Note: Untuk menghindari log data yang terlalu besar, hanya nilai sample pertama (index 0)
    yang dijadikan representasi.
    """
    n_samples, n_features = X.shape
    print(f"n_samples: {n_samples}, n_features: {n_features}")
    n_classes = 10

    # Inisialisasi bobot dan bias
    W = np.random.randn(n_features, n_classes) * 0.01
    b = np.zeros((1, n_classes))

    # Ubah label ke format one-hot
    y_encoded = one_hot_encode(y, n_classes)
    # print(f"y_encoded: {y_encoded}")
    # print(f"y_encoded shape: {y_encoded.shape}")

    # List untuk menyimpan log per epoch
    log_data = []

    for epoch in range(epochs):
        # Forward pass
        Z = np.dot(X, W) + b       # net input
        A = softmax(Z)             # fnet, output probabilitas

        # Hitung loss cross-entropy
        loss = -np.sum(y_encoded * np.log(A + 1e-8)) / n_samples

        # Backpropagation
        dZ = A - y_encoded         # error (dz)
        dW = np.dot(X.T, dZ) / n_samples   # gradien bobot
        db = np.sum(dZ, axis=0, keepdims=True) / n_samples  # gradien bias

        # Update parameter
        W -= learning_rate * dW
        b -= learning_rate * db

        # Simpan log untuk epoch ini, ambil nilai sample pertama sebagai contoh
        log_data.append({
            "Epoch": epoch,
            "X": np.array2string(X[0], precision=2, separator=','),
            "Y_Label": y[0],
            "W": np.array2string(W, precision=4, separator=','),
            "b": np.array2string(b, precision=4, separator=','),
            "net": np.array2string(Z[0], precision=4, separator=','),
            "fnet": np.array2string(A[0], precision=4, separator=','),
            "dw": np.array2string(dW, precision=4, separator=','),
            "dz": np.array2string(dZ[0], precision=4, separator=','),
            "db": np.array2string(db, precision=4, separator=',')
        })

        if epoch % 10 == 0:
            print(f"Epoch {epoch}/{epochs}, Loss: {loss:.4f}")

    # Simpan log training ke file CSV
    # Tambahkan timestamp ke nama file log
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    csv_filename = f"training_log_detailed_{timestamp}.csv"

    with open(csv_filename, 'w', newline='') as csvfile:
        fieldnames = ["Epoch", "X", "Y_Label", "W", "b", "net", "fnet", "dw", "dz", "db"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in log_data:
            writer.writerow(row)
    print(f"Detailed log training disimpan ke file {csv_filename}")

    # Definisikan fungsi net dan netfunction untuk keperluan prediksi
    def net(x):
        return np.dot(x, W) + b

    def netfunction(x):
        z = net(x)
        return softmax(z)

    # Kembalikan model
    model = {
        'W': W,
        'b': b,
        'net': net,
        'netfunction': netfunction
    }
    return model



In [4]:
# Memuat data dari folder dataset
X, y = load_data()
print("Data berhasil dimuat. Shape X:", X.shape, ", Shape y:", y.shape)

# Training model dengan parameter tertentu (misal: learning_rate=0.1, epochs=100)
model = train_nn(X, y, learning_rate=0.1, epochs=100)

img shape (100, 100, 3)
img flat : (30000,)
Data berhasil dimuat. Shape X: (1097, 30000) , Shape y: (1097,)
n_samples: 1097, n_features: 30000
      y   y0   y1   y2   y3   y4   y5   y6   y7   y8   y9
0     0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1     0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2     0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3     0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4     0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
...  ..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
1092  9  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
1093  9  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
1094  9  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
1095  9  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
1096  9  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

[1097 rows x 11 columns]
Epoch 0/100, Loss: 2.3728
Epoch 10/100, Loss: 2.9648
Epoch 20/100, Loss: 0.3956
Epoch 30/100, Loss: 0.1981
Epoch 40/100, Loss: 0.1352
E

KeyboardInterrupt: 

In [5]:
import numpy as np
import pandas as pd
import datetime

predicted_classes = []
actual_classes = []

# Lakukan prediksi untuk 1000 data pertama
for i in range(1000):
    sample = X[i].reshape(1, -1)
    # Hitung nilai linear (net) dan output softmax (fnet)
    net_value = model['net'](sample)
    fnet_value = model['netfunction'](sample)
    # Prediksi kelas: indeks dengan nilai tertinggi
    predicted_class = np.argmax(fnet_value)
    
    predicted_classes.append(predicted_class)
    # Asumsikan y merupakan label aktual untuk data X
    actual_classes.append(y[i])

# Membuat DataFrame untuk menampilkan tabel perbandingan
df = pd.DataFrame({
    'Predicted Class': predicted_classes,
    'Actual Class': actual_classes
})

# Menambahkan kolom Error: bernilai True jika prediksi salah, dan False jika benar
df['Error'] = df['Predicted Class'] != df['Actual Class']

# Atau jika ingin menampilkan sebagai teks "Correct" dan "Incorrect"
# df['Error'] = df.apply(lambda row: 'Incorrect' if row['Predicted Class'] != row['Actual Class'] else 'Correct', axis=1)

# Menghitung akurasi prediksi dalam persentase
accuracy = (df['Predicted Class'] == df['Actual Class']).mean() * 100
print(f"Akurasi prediksi: {accuracy:.2f}%")

# Menampilkan DataFrame
print(df)

# Menyimpan tabel ke file CSV dengan timestamp
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
csv_filename = f"prediction_comparison_{timestamp}.csv"
df.to_csv(csv_filename, index=False)
print(f"CSV file telah disimpan dengan nama: {csv_filename}")

# Print input shape
print("Input shape: ", X.shape)
print("Image shape: ", X[0].shape)

save_model(model, "model_seratus.npz")
print("Bias shape: ", model['b'].shape)
print("Weight shape: ", model['W'].shape)

# Simpan nilai prediksi (net dan fnet) ke file teks
with open("prediction_details.txt", "w") as f:
    f.write("Net (nilai linear):\n")
    f.write(np.array2string(net_value, precision=4, separator=', ') + "\n\n")
    f.write("fnet (hasil softmax):\n")
    f.write(np.array2string(fnet_value, precision=4, separator=', ') + "\n")
    # Write nilai weight dan bias
    f.write("\nWeight:\n")
    f.write(np.array2string(model['W'], precision=4, separator=', ') + "\n\n")
    f.write("Bias:\n")
    f.write(np.array2string(model['b'], precision=4, separator=', ') + "\n")
print("Detail prediksi disimpan ke file prediction_details.txt")


NameError: name 'model' is not defined

Load Model dan Klasifikasi

In [13]:
import cv2
import numpy as np
def load_model(filename="model.npz"):
    """
    Memuat model dari file NPZ.
    Mengembalikan dictionary dengan weight (W), bias (b),
    dan fungsi net serta netfunction untuk prediksi.
    """
    data = np.load(filename)
    W = data['W']
    b = data['b']
    print(f"W shape: {W.shape}, b shape: {b.shape}")
    print(f"W : {W}, \nb : {b}")
    
    def softmax(z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        # print(f"exp_z: {exp_z} np.max: {np.max(z, axis=1, keepdims=True)} np.sum: {np.sum(exp_z, axis=1, keepdims=True)}") 
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)
    
    def net(x):
        return np.dot(x, W) + b

    def netfunction(x):
        # Pastikan fungsi softmax sudah didefinisikan
        return softmax(net(x))
    
    model = {
        'W': W,
        'b': b,
        'net': net,
        'netfunction': netfunction
    }
    print(f"Model berhasil dimuat dari file {filename}")
    return model



In [14]:
model = load_model("model_seratus.npz")
img = cv2.imread("Datasets/SizeSeratus/numbers/mnist_png/Sample2/5.png")
if img is None:
    print("File tidak ditemukan.")
img_flat = img.flatten().astype(np.float32) / 255.0
sample = img_flat.reshape(1, -1)
net_value = model['net'](sample)
fnet_value = model['netfunction'](sample)
predicted_class = np.argmax(fnet_value)
print("Prediksi kelas untuk sample pertama:", predicted_class)

W shape: (30000, 10), b shape: (1, 10)
W : [[-0.01160494 -0.00067689 -0.01154553 ... -0.00175372  0.00881999
  -0.03306866]
 [-0.00281772  0.0013718  -0.00275804 ...  0.00392029 -0.00597277
   0.00153628]
 [ 0.00019685 -0.00216679  0.00134295 ...  0.0054345   0.00723155
  -0.01078134]
 ...
 [ 0.00191728  0.00543371  0.00407294 ... -0.01224739  0.00300085
   0.01270108]
 [ 0.00150216 -0.00943852  0.00069959 ...  0.01654285 -0.00350291
   0.0005638 ]
 [-0.01157613  0.00212678  0.01079763 ...  0.00844008  0.00146719
   0.00818699]], 
b : [[-0.0275864   0.00762513  0.00618943  0.00119782  0.00268268  0.03942614
  -0.01084269  0.01537126 -0.0449403   0.01087693]]
Model berhasil dimuat dari file model_seratus.npz
Prediksi kelas untuk sample pertama: 2


# OTHERS

In [ ]:
import cv2
import numpy

# Read the image file
img = cv2.imread('Datasets/Sample0/1.png')

# Print the data matrix
print(img)

None


In [ ]:
import cv2


img = cv2.imread('Datasets/Sample0/1.png')
# img = cv2.imread('coba.png', cv2.IMREAD_GRAYSCALE)
# img = cv2.imread('coba.png')

# print 5 rows of the image
print(img[:5])
print(f"image shape : {img.shape} ")
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

TypeError: 'NoneType' object is not subscriptable

In [ ]:
import cv2
import numpy as np

# --- Membaca dan menampilkan image dari Datasets/Sample0/1.png ---
img0 = cv2.imread('Datasets/Sample0/1.png')
if img0 is None:
    print("Image tidak ditemukan: Datasets/Sample0/1.png")
else:
    print("Image dari Datasets/Sample0/1.png:")
    print(img0)
    print("5 baris pertama dari image:")
    print(img0[:5])
    print(f"Shape image: {img0.shape}")
    cv2.imshow('Image Sample0', img0)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# --- Membaca dan menampilkan image dari Datasets/Sample1/1.png ---
img1 = cv2.imread('Datasets/Sample1/1.png')
if img1 is None:
    print("Image tidak ditemukan: Datasets/Sample1/1.png")
else:
    print("Image dari Datasets/Sample1/1.png:")
    print(img1)
    print("5 baris pertama dari image:")
    print(img1[:5])
    print(f"Shape image: {img1.shape}")
    cv2.imshow('Image Sample1', img1)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# --- Contoh nilai Net dan fnet untuk prediksi ---
Net = np.array([[ 5.9158, -3.0557, -1.3257,  0.3136, -2.8025,  1.6988,  0.7880, -0.6799,  0.1001, -0.5342]])


# Melakukan prediksi dengan mencari indeks nilai maksimum di fnet
prediction = np.argmax(fnet, axis=1)
print(f"Predicted class: {prediction[0]}")


Predicted class: 0


In [ ]:
import os

def load_model(filename="model.npz"):
    """
    Memuat model dari file NPZ.
    Mengembalikan dictionary dengan weight (W), bias (b),
    dan fungsi net serta netfunction untuk prediksi.
    """
    data = np.load(filename)
    W = data['W']
    b = data['b']
    
    def net(x):
        return np.dot(x, W) + b

    def netfunction(x):
        # Pastikan fungsi softmax sudah didefinisikan
        return softmax(net(x))
    
    model = {
        'W': W,
        'b': b,
        'net': net,
        'netfunction': netfunction
    }
    print(f"Model berhasil dimuat dari file {filename}")
    return model

model = load_model("model.npz")

# Get the first indexed file with .png format in the directory
directory = 'Datasets/Sample4'
files = [f for f in os.listdir(directory) if f.endswith('.png')]
files.sort()  # Ensure the files are sorted
if files:
    img_path = os.path.join(directory, files[0])
    img = cv2.imread(img_path)
else:
    img = None
if img is None:
    print("Image tidak ditemukan: Datasets/Sample0/1.png")
else:
    print(f"Image shape: {img.shape}")
    # Resize image menjadi 28x28
    img = cv2.resize(img, (28, 28))
    # Ubah ke grayscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Ubah ke format float32
    img = img.astype(np.float32)
    # Normalisasi nilai pixel menjadi 0-1
    img = img / 255.0
    # Flatten image menjadi vektor
    img = img.flatten()
    print(f"img : {img}")
    print(f"img setelah diubah: {img.shape}")
    # Melakukan prediksi dengan model
    fnet = model['net'](img)
    prediction = np.argmax(fnet)
    print(f"Predicted class: {prediction}")


Model berhasil dimuat dari file model.npz
Image shape: (28, 28, 3)
img : [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.      